In [1]:
import warnings
warnings.filterwarnings('ignore')

# 1. Base case tokenization

In [3]:
from transformers import DistilBertTokenizer

sentence = 'happy'

tokenizer_ckpt = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_ckpt)

In [4]:
tokenizer.tokenize(sentence)

['happy']

# 2. Encryption (AES-256)

In [5]:
import os
import base64

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend

In [7]:
def generate_key(password: str, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())

In [9]:
password = "securepassword"
salt = os.urandom(16)
iv = os.urandom(16)

key = generate_key(password, salt)
key

b'\xe3h\x85\x7f\x18\xab\xa7\xeer\xf8K\xec\x91*\x0e\x9b\x86\xa2\x9f\x04rxR\x80\t\x9e\x1d \x89\x0b\xfei'

In [10]:
def encrypt(data: str, key: bytes, iv: bytes) -> str:
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    
    padder = PKCS7(128).padder()
    padded_data = padder.update(data.encode()) + padder.finalize()
    
    encrypted = encryptor.update(padded_data) + encryptor.finalize()
    return base64.b64encode(encrypted).decode()

In [14]:
plaintext = tokenizer.tokenize(sentence)[0]
plaintext

'happy'

In [15]:
encrypted = encrypt(plaintext, key, iv)
encrypted

'Y3S3xqiXfnx8WsUWiHF3Cw=='

In [16]:
def decrypt(encrypted_data: str, key: bytes, iv: bytes) -> str:
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    
    encrypted_data = base64.b64decode(encrypted_data)
    decrypted = decryptor.update(encrypted_data) + decryptor.finalize()
    
    unpadder = PKCS7(128).unpadder()
    unpadded_data = unpadder.update(decrypted) + unpadder.finalize()
    
    return unpadded_data.decode()

In [17]:
decrypted = decrypt(encrypted, key, iv)
decrypted

'happy'